# מטלה 2 - למידת מכונה

In [1]:
import pandas as pd
import warnings
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, make_scorer

In [2]:
df = pd.read_csv('C:/Users/Tahel Hermon/Downloads/dataset.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   manufactor       1500 non-null   object 
 1   Year             1500 non-null   int64  
 2   model            1500 non-null   object 
 3   Hand             1500 non-null   int64  
 4   Gear             1499 non-null   object 
 5   capacity_Engine  1474 non-null   object 
 6   Engine_type      1495 non-null   object 
 7   Prev_ownership   771 non-null    object 
 8   Curr_ownership   772 non-null    object 
 9   Area             1351 non-null   object 
 10  City             1500 non-null   object 
 11  Price            1500 non-null   float64
 12  Pic_num          1476 non-null   float64
 13  Cre_date         1500 non-null   object 
 14  Repub_date       1500 non-null   object 
 15  Description      1500 non-null   object 
 16  Color            730 non-null    object 
 17  Km            

#### טיפול בערכים חסרים

In [4]:
df2 = df.copy()
df3 = df2.drop_duplicates()
len(df3)

1372

##### המרת עמודת ק"מ

In [5]:
def km(df):
    df['Km'] = pd.to_numeric(df['Km'], errors='coerce')
    df['Km'] = df['Km'].astype('float64')
    df['Km'] = df['Km'].fillna(df['Km'].mean())

##### מילוי ערכים חסרים

In [6]:
import warnings
warnings.filterwarnings("ignore")
def filling(df):
    km(df)
    df['capacity_Engine'] = df['capacity_Engine'].fillna(df['capacity_Engine'].mode()[0])
    df['capacity_Engine'] = df['capacity_Engine'].str.strip().str.replace(',', '')
    df['Engine_type'] = df['Engine_type'].fillna(df['Engine_type'].mode()[0])
    df['Year'] = df['Year'].fillna(df['Year'].mode()[0])
    df['Hand'] = df['Hand'].fillna(df['Hand'].mode()[0])
    df['Gear'] = df['Gear'].fillna(df['Gear'].mode()[0])
    df['Pic_num'] = df['Pic_num'].fillna('0')
    df['Area'] = df['Area'].fillna('אין אזור מוגדר')

##### הורדת עמודות עם פחות מ50% נתונים לפי כלל אצבע

In [7]:
def low_data(df):
    threshold=0.5
    threshold_count = int(threshold * len(df))
    df.dropna(axis = 1,thresh=threshold_count, inplace=True)

In [8]:
km(df3)
low_data(df3)

#### טיפול בחריגים

In [9]:
import numpy as np
def Clearing_Outliers_By_IQR(df, colName, lp, up):
    Q1 = np.percentile(df[colName], lp)
    Q3 = np.percentile(df[colName], up)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[colName] < lower_bound) | (df[colName] > upper_bound)][colName]
    print(f"{outliers.min()} , {outliers.max()}")
    df[colName][(df[colName] < lower_bound) | (df[colName] > upper_bound)] = np.nan
    print(f'now have {df[colName].isna().sum()} nans')

In [10]:
Clearing_Outliers_By_IQR(df3, 'Year', 30, 99)
Clearing_Outliers_By_IQR(df3, 'Km', 0, 75)
Clearing_Outliers_By_IQR(df3, 'Hand', 0, 64)

1983 , 2000
now have 11 nans
400000.0 , 1550000.0
now have 8 nans
7 , 10
now have 16 nans


In [11]:
filling(df3)
df3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1372 entries, 0 to 1499
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   manufactor       1372 non-null   object 
 1   Year             1372 non-null   float64
 2   model            1372 non-null   object 
 3   Hand             1372 non-null   float64
 4   Gear             1372 non-null   object 
 5   capacity_Engine  1372 non-null   object 
 6   Engine_type      1372 non-null   object 
 7   Area             1372 non-null   object 
 8   City             1372 non-null   object 
 9   Price            1372 non-null   float64
 10  Pic_num          1372 non-null   object 
 11  Cre_date         1372 non-null   object 
 12  Repub_date       1372 non-null   object 
 13  Description      1372 non-null   object 
 14  Km               1372 non-null   float64
dtypes: float64(4), object(11)
memory usage: 171.5+ KB


### Feature Engineering

#### Km

In [35]:
def feature_bins(df):
    bins = pd.cut(df['Km'], bins=10, labels=False)
    df['Km_binned'] = bins

In [36]:
def map_area_to_region(area):
    area_bins = {
        'אזור המרכז': ['תל אביב', 'ראשון לציון', 'רמת גן', 'גבעתיים', 'חולון', 'בת ים', 'פתח תקוה והסביבה', 'בקעת אונו', 'מודיעין והסביבה', 'ראש העין והסביבה'],
        'שרון': ['רעננה - כפר סבא', 'מושבים בשרון', 'רמת השרון - הרצליה', 'הוד השרון והסביבה', 'נתניה והסביבה', 'קיסריה והסביבה', 'חדרה וישובי עמק חפר'],
        'דרום': ['אשדוד - אשקלון', 'באר שבע והסביבה', 'אילת והערבה', 'מושבים בדרום', 'מושבים בשפלה'],
        'צפון': ['חיפה וחוף הכרמל', 'קריות', 'עכו - נהריה', 'טבריה והסביבה', 'גליל ועמקים', 'עמק יזרעאל', 'כרמיאל והסביבה', 'זכרון - בנימינה', 'יישובי השומרון'],
        'בקעה ושפלה': ['נס ציונה - רחובות', 'ראשל"צ והסביבה', 'גדרה יבנה והסביבה', 'רמלה - לוד', 'בית שמש והסביבה', 'מושבים במרכז']
    }
    
    for region, areas in area_bins.items():
        if any(area.startswith(a) for a in areas):
            return region
    return 'אזור לא מוגדר'

In [37]:
def feature_area(df):
    df['Region'] = df['Area'].apply(map_area_to_region)

In [38]:
def feature_gear(df):
    df_gear =  df.copy()
    df_gear = df_gear.loc[:,['Gear']]
    gear_encoder = OneHotEncoder(min_frequency=int(0.45 * len(df)), sparse_output=False)
    gear_encoded = gear_encoder.fit_transform(df_gear)
    gear_feature_names = gear_encoder.get_feature_names_out(df_gear.columns)
    gear_encoded = pd.DataFrame(gear_encoded, columns=gear_feature_names, index=df_gear.index)
    return gear_encoded

In [39]:
def feature_engine(df):
    df_engine_type = df.copy()
    df_engine_type = df_engine_type.loc[:,['Engine_type']]
        
    df_engine_type_encoder = OneHotEncoder(min_frequency=int(0.45 * len(df)), sparse_output=False)
    engine_type_encoded = df_engine_type_encoder.fit_transform(df_engine_type)
    engine_type_feature_names = df_engine_type_encoder.get_feature_names_out(df_engine_type.columns)
    engine_type_encoded = pd.DataFrame(engine_type_encoded, columns=engine_type_feature_names, index=df_engine_type.index)
    
    return engine_type_encoded

In [40]:
def feature_manufactor(df):
    df_manufactor = df.copy()
    df_manufactor = df_manufactor.loc[:,['manufactor']]

    unique_count = df['manufactor'].nunique()
    df_manufactor_encoder = OneHotEncoder(min_frequency=int(len(df)/unique_count), sparse_output=False)
    manufactor_encoded = df_manufactor_encoder.fit_transform(df_manufactor)
    manufactor_feature_names = df_manufactor_encoder.get_feature_names_out(df_manufactor.columns)
    manufactor_encoded = pd.DataFrame(manufactor_encoded, columns=manufactor_feature_names, index=df_manufactor.index)

    return manufactor_encoded

In [41]:
def feature_region(df):
    df_region = df.copy()
    df_region = df_region.loc[:,['Region']]

    unique_count = df['Region'].nunique()    
    df_region_encoder = OneHotEncoder(min_frequency=int(len(df)/unique_count), sparse_output=False)
    region_encoded = df_region_encoder.fit_transform(df_region)
    region_feature_names = df_region_encoder.get_feature_names_out(df_region.columns)
    region_encoded = pd.DataFrame(region_encoded, columns=region_feature_names, index=df_region.index)
    return region_encoded

In [42]:
def feature_km(df):
    df_Km_binned = df.copy()
    df_Km_binned = df_Km_binned.loc[:,['Km_binned']]
    df_Km_binned_encoder = OneHotEncoder(min_frequency=int(len(df)/10), sparse_output=False)
    Km_binned_encoded = df_Km_binned_encoder.fit_transform(df_Km_binned)
    Km_binned_feature_names = df_Km_binned_encoder.get_feature_names_out(df_Km_binned.columns)
    Km_binned_encoded = pd.DataFrame(Km_binned_encoded, columns=Km_binned_feature_names, index=df_Km_binned.index)
    return Km_binned_encoded

In [43]:
def feature_rank(df):
    df_rank = df.copy()
    numeric_features = ['Year', 'Hand', 'capacity_Engine', 'Km']
    
    scaler = StandardScaler()
    df_rank[numeric_features] = scaler.fit_transform(df_rank[numeric_features])
    df_rank['Rank'] = df_rank[numeric_features].sum(axis=1)
    df_rank['Rank'] = 1 + 99 * (df_rank['Rank'] - df_rank['Rank'].min()) / (df_rank['Rank'].max() - df_rank['Rank'].min())
    df_rank['Rank'] = df_rank['Rank'].round(2)
    return df_rank

In [51]:
def prepare_data(df):
    feature_area(df)
    feature_bins(df)
    gear_encoded = feature_gear(df)
    engine_type_encoded = feature_engine(df)
    manufactor_encoded = feature_manufactor(df)
    region_encoded = feature_region(df)
    km_encoded = feature_km(df)
    df_rank = feature_rank(df)
    
    df = pd.concat([df, gear_encoded, engine_type_encoded, manufactor_encoded,
                    region_encoded, km_encoded, df_rank['Rank']], axis=1)
    
    df['Km_per_Year'] = df['Km'] / df['Year']
    
    return df

In [55]:
df3 = prepare_data(df3)

In [56]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1372 entries, 0 to 1499
Data columns (total 44 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   manufactor                      1372 non-null   object 
 1   Year                            1372 non-null   float64
 2   model                           1372 non-null   object 
 3   Hand                            1372 non-null   float64
 4   Gear                            1372 non-null   object 
 5   capacity_Engine                 1372 non-null   object 
 6   Engine_type                     1372 non-null   object 
 7   Area                            1372 non-null   object 
 8   City                            1372 non-null   object 
 9   Price                           1372 non-null   float64
 10  Pic_num                         1372 non-null   object 
 11  Cre_date                        1372 non-null   object 
 12  Repub_date                      1372 no

### Elastic Net

In [80]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def model(df):
    X_dropped = df[['Price','manufactor', 'Gear', 'model', 'Engine_type','Area', 'City', 'Price', 'Pic_num', 'Cre_date', 
        'Repub_date', 'Description', 'Region']]
    X = df.drop(columns = ['Price','manufactor', 'Gear', 'model', 'Engine_type','Area', 'City', 'Price', 'Pic_num', 'Cre_date', 
        'Repub_date', 'Description', 'Region'])
    y = df['Price']

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    X_scaled_df = pd.DataFrame(X_scaled,columns=X.columns)
    X_final = pd.concat([X_scaled_df, X_dropped.reset_index(drop=True)], axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.2, random_state=42)
    
    param_grid = {
        'alpha': [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0],
        'l1_ratio': [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
    }
    
    elastic_net = ElasticNet(random_state=42)
    rmse_scorer = make_scorer(rmse, greater_is_better=False)
    scores = cross_val_score(elastic_net, X_scaled_df, y, cv=10, scoring=rmse_scorer)    
    initial_best_score = np.max(scores)  # Use min because lower RMSE is better

    
    grid_search = GridSearchCV(estimator=ElasticNet(random_state=42), param_grid=param_grid, scoring=rmse_scorer, cv=10)
    grid_search.fit(X_scaled_df, y)
    grid_best_score = np.max(grid_search.cv_results_['mean_test_score'])
    
    
    best_model = ElasticNet(**grid_search.best_params_, random_state=42)
    best_model.fit(X_scaled_df, y) 
    best_scores = cross_val_score(best_model, X_scaled_df, y, cv=10, scoring=rmse_scorer)
    print("Best model CV scores (RMSE):", abs(np.max(best_scores)))
    print("Best parameters:", grid_search.best_params_)
    
    
    coef_abs = abs(best_model.coef_) 
    feature_names = X_scaled_df.columns
    feature_importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Coefficient': best_model.coef_,
        'num_Coefficient': coef_abs
    })
    
    
    sorted_features = feature_importance_df.sort_values(by='num_Coefficient', ascending=False)
    top_features = sorted_features.drop(columns = ['num_Coefficient']).head(5).reset_index(drop=True)
    print("Top 5 Most Impactful Features:")
    print(top_features)

In [81]:
model(df3)

Best model CV scores (RMSE): 14318.3603489251
Best parameters: {'alpha': 0.1, 'l1_ratio': 0.8}
Top 5 Most Impactful Features:
                        Feature  Coefficient
0                          Year  9238.712426
1                          Rank  4022.590358
2                          Hand -3277.696417
3  Km_binned_infrequent_sklearn -2917.959994
4             manufactor_טויוטה  2670.515371


In [82]:
df3['Km_binned']

0       3
1       1
2       3
3       0
4       1
       ..
1495    0
1496    4
1497    3
1498    3
1499    4
Name: Km_binned, Length: 1372, dtype: int64